# Doc2Vec Tutorial on the Lee Dataset

In [1]:
import gensim
import os
import collections
import random

## What is it?

Doc2Vec is an NLP tool for representing documents as a vector and is a generalizing of the Word2Vec method. This tutorial will serve as an introduction to Doc2Vec and present ways to train and assess a Doc2Vec model. Insted of dealing with non correlated set of sentences (as we did in Word2Vec) , we deal with a set of correlated sentences(preferably from the same paragraph).

## Resources

* [Word2Vec Paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [Doc2Vec Paper](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)
* [Dr. Michael D. Lee's Website](http://faculty.sites.uci.edu/mdlee)
* [Lee Corpus](http://faculty.sites.uci.edu/mdlee/similarity-data/)
* [IMDB Doc2Vec Tutorial](https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb)

## Getting Started

To get going, we'll need to have a set of documents to train our doc2vec model. In theory, a document could be anything from a short 140 character tweet, a single paragraph (i.e., journal article abstract), a news article, or a book. In NLP parlance a collection or set of documents is often referred to as a <b>corpus</b>. 

For this tutorial, we'll be training our model using the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting
Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.

And we'll test our model by eye using the much shorter [Lee Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) which contains 50 documents.

In [3]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

## Define a Function to Read and Preprocess Text

Below, we define a function to open the train/test file (with latin encoding), read the file line-by-line, pre-process each line using a simple gensim pre-processing tool (i.e., tokenize text into individual words, remove punctuation, set to lowercase, etc), and return a list of words. Note that, for a given file (aka corpus), each continuous line constitutes a single document and the length of each line (i.e., document) can vary. Also, to train the model, we'll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply the zero-based line number.

In [4]:
import io
def read_corpus(fname, tokens_only=False):
    with io.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [5]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

Let's take a look at the training corpus

In [6]:
train_corpus[:2]

[TaggedDocument(words=[u'hundreds', u'of', u'people', u'have', u'been', u'forced', u'to', u'vacate', u'their', u'homes', u'in', u'the', u'southern', u'highlands', u'of', u'new', u'south', u'wales', u'as', u'strong', u'winds', u'today', u'pushed', u'huge', u'bushfire', u'towards', u'the', u'town', u'of', u'hill', u'top', u'new', u'blaze', u'near', u'goulburn', u'south', u'west', u'of', u'sydney', u'has', u'forced', u'the', u'closure', u'of', u'the', u'hume', u'highway', u'at', u'about', u'pm', u'aedt', u'marked', u'deterioration', u'in', u'the', u'weather', u'as', u'storm', u'cell', u'moved', u'east', u'across', u'the', u'blue', u'mountains', u'forced', u'authorities', u'to', u'make', u'decision', u'to', u'evacuate', u'people', u'from', u'homes', u'in', u'outlying', u'streets', u'at', u'hill', u'top', u'in', u'the', u'new', u'south', u'wales', u'southern', u'highlands', u'an', u'estimated', u'residents', u'have', u'left', u'their', u'homes', u'for', u'nearby', u'mittagong', u'the', u'ne

And the testing corpus looks like this:

In [7]:
print(test_corpus[:2])

[[u'the', u'national', u'executive', u'of', u'the', u'strife', u'torn', u'democrats', u'last', u'night', u'appointed', u'little', u'known', u'west', u'australian', u'senator', u'brian', u'greig', u'as', u'interim', u'leader', u'shock', u'move', u'likely', u'to', u'provoke', u'further', u'conflict', u'between', u'the', u'party', u'senators', u'and', u'its', u'organisation', u'in', u'move', u'to', u'reassert', u'control', u'over', u'the', u'party', u'seven', u'senators', u'the', u'national', u'executive', u'last', u'night', u'rejected', u'aden', u'ridgeway', u'bid', u'to', u'become', u'interim', u'leader', u'in', u'favour', u'of', u'senator', u'greig', u'supporter', u'of', u'deposed', u'leader', u'natasha', u'stott', u'despoja', u'and', u'an', u'outspoken', u'gay', u'rights', u'activist'], [u'cash', u'strapped', u'financial', u'services', u'group', u'amp', u'has', u'shelved', u'million', u'plan', u'to', u'buy', u'shares', u'back', u'from', u'investors', u'and', u'will', u'raise', u'milli

Notice that the testing corpus is just a list of lists and does not contain any tags.

## Training the Model

### Instantiate a Doc2Vec Object 

Now, we'll instantiate a Doc2Vec model with a vector size with 50 words and iterating over the training corpus 10 times. We set the minimum word count to 2 in order to give higher frequency words more weighting. Model accuracy can be improved by increasing the number of iterations but this generally increases the training time.

In [18]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=10)

### Build a Vocabulary

In [19]:
model.build_vocab(train_corpus)

Essentially, the vocabulary is a dictionary (accessible via `model.vocab`) of all of the unique words extracted from the training corpus along with the count (e.g., `model.vocab['penalty'].count` for counts for the word `penalty`).

### Time to Train

This should take no more than 2 minutes

In [20]:
%time model.train(train_corpus)

CPU times: user 1.73 s, sys: 52 ms, total: 1.78 s
Wall time: 724 ms


426871

### Inferring a Vector

One important thing to note is that you can now infer a vector for any piece of text without having to re-train the model by passing a list of words to the `model.infer_vector` function. This vector can then be compared with other vectors via cosine similarity.

In [26]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([ 0.01430994,  0.00786653,  0.00154077,  0.01312404,  0.00389613,
        0.01039447,  0.02234556, -0.0270298 , -0.01744372,  0.00116681,
        0.02232473, -0.03406299, -0.02925538, -0.02054195, -0.01678725,
        0.00577997,  0.01852101, -0.01274862,  0.00999305, -0.00151929,
       -0.00562159, -0.02457337,  0.0394986 ,  0.00920296,  0.00802251,
        0.01436673, -0.01646674,  0.01271828, -0.03088115, -0.02209166,
       -0.01152834,  0.0499837 ,  0.01577185, -0.00595656,  0.01954435,
       -0.00160794, -0.0021305 , -0.01472405, -0.0050831 , -0.0361198 ,
       -0.03533225,  0.01583265, -0.0089896 , -0.02636493, -0.01778116,
       -0.02680507,  0.03331323, -0.00059   ,  0.00202905,  0.01007229], dtype=float32)

## Assessing Model

To assess our new model, we'll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we're pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we've likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we'll keep track of the second ranks for a comparison of less similar documents. 

In [27]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

Let's count how each document ranks with respect to the training corpus 

In [28]:
collections.Counter(ranks)  #96% accuracy

Counter({0: 33,
         1: 19,
         2: 22,
         3: 16,
         4: 8,
         5: 17,
         6: 17,
         7: 12,
         8: 11,
         9: 6,
         10: 6,
         11: 4,
         12: 4,
         13: 8,
         14: 4,
         15: 2,
         16: 3,
         17: 7,
         18: 6,
         19: 3,
         20: 4,
         21: 5,
         22: 2,
         24: 6,
         25: 3,
         26: 2,
         27: 3,
         28: 1,
         29: 4,
         30: 2,
         31: 2,
         32: 2,
         33: 2,
         34: 2,
         35: 3,
         36: 2,
         37: 2,
         38: 1,
         39: 2,
         40: 3,
         41: 1,
         42: 1,
         45: 3,
         49: 2,
         50: 2,
         53: 1,
         54: 2,
         57: 1,
         60: 1,
         63: 2,
         64: 1,
         65: 1,
         68: 1,
         71: 1,
         72: 1,
         78: 1,
         89: 1,
         93: 2,
         95: 1,
         97: 1,
         100: 1,
         101: 1,
        

Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document. This is great and not entirely surprising. We can take a look at an example:

In [14]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Notice above that the most similar document is has a similarity score of ~80% (or higher). However, the similarity score for the second ranked documents should be significantly lower (assuming the documents are in fact different) and the reasoning becomes obvious when we examine the text itself

In [15]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (24): «team of police is currently escorting two swiss tourists back to the safety of central australian community after their vehicle sank in sand in the finke river overnight police spokesman says police were called to the area kilometres west of alice springs after an emergency beacon was activated and received by australian search and rescue the tourists had tried to cross the river when their vehicle became submerged in soft sand the spokesman says ground unit also attended but police officers had to walk the final kilometres to reach the stranded swiss nationals who were stuck in the vehicle the tourists and the rescue team are expected to walk for about four hours this morning before driving to the hermansburg community»

Similar Document (54, 0.9989227652549744): «police are interviewing year old man for stealing car with child inside from the northside shopping centre in alice springs senior sergeant michael potts says the month old boy was left on elliot street

## Testing the Model

Using the same approach above, we'll infer the vector for a randomly chosen test document, and compare the document to our model by eye.

In [16]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (26): «how did allegedly unregistered missile warheads come to be stored on canadian businessman anti terrorism training facility in new mexico and canadian officials are still trying to figure that out but one security expert says the mystery is chilling one david hudak was arrested in the united states more than week ago when according to court documents agents searching his property found the warheads stored in crates that were marked charge demolition»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (153, 0.9980329275131226): «at least two helicopters have landed near tora bora mountain in eastern afghanistan in what could be the start of raid against al qaeda fighters an afp journalist said the helicopters landed around pm local time am aedt few hours after al qaeda fighters rejected deadline set by afghan militia leaders for them to surrender or face death us warplanes have been bombing the network of caves and tunnels for eight days 

### Wrapping Up

That's it! Doc2Vec is a great way to explore relationships between documents.

Developer - Pranav Shukla

email id- pranavdynamic@gmail.com